## Comparison of CNN Models for Mass Detection in Mammography Images

### Setup

First we install any necessary packages.

In [11]:
!pip install pydicom

Then we load the packages.

In [0]:
import os
import pydicom
import cv2
import numpy as np
import matplotlib.pyplot as plt
import keras as K
import pandas as pd

We load the [INBreast](https://github.com/wentaozhu/deep-mil-for-whole-mammogram-classification/issues/12) dataset from Dropbox.

In [13]:
!wget -O inbreast https://www.dropbox.com/sh/adc55725ksmc58u/AABa6KRTs_Pnd0S6u2bTvMlva?dl=0 inbreast
!unzip inbreast

--2020-04-14 13:40:46--  https://www.dropbox.com/sh/adc55725ksmc58u/AABa6KRTs_Pnd0S6u2bTvMlva?dl=0
Resolving www.dropbox.com (www.dropbox.com)... 162.125.3.1, 2620:100:601b:1::a27d:801
Connecting to www.dropbox.com (www.dropbox.com)|162.125.3.1|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /sh/raw/adc55725ksmc58u/AABa6KRTs_Pnd0S6u2bTvMlva [following]
--2020-04-14 13:40:46--  https://www.dropbox.com/sh/raw/adc55725ksmc58u/AABa6KRTs_Pnd0S6u2bTvMlva
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://ucb1fb13774ba2750467d38016ce.dl.dropboxusercontent.com/zip_download_get/AZPlc0GzjJeL1CsrciOOqQ2_GT7cHI4HURo9wrmel6JzQUwErHxvv7xe0NmHLLa5CEgt4-QFbyMYpTwhaaGezcwumeHTLThgoDzSPoywIUdW_w [following]
--2020-04-14 13:40:48--  https://ucb1fb13774ba2750467d38016ce.dl.dropboxusercontent.com/zip_download_get/AZPlc0GzjJeL1CsrciOOqQ2_GT7cHI4HURo9wrmel6JzQUwErHxvv7xe0NmHLLa5CEgt4-QFbyMYpTwhaaGe

In [14]:
!unzip inbreast inbreast

Archive:  inbreast
caution: filename not matched:  inbreast


In [17]:
inbreast_dir = './'

inbreast = []
for file in os.listdir(inbreast_dir):
  if file.endswith(".dcm"):
    dataset = pydicom.dcmread(os.path.join(inbreast_dir, file))
    inbreast.append(dataset)

inbreast_labels = pd.read_csv(os.path.join(inbreast_dir, 'INbreast.csv'))

print('Finished loading INBreast data.')

FileNotFoundError: ignored

In [9]:
def preprocess(img):
  result = cv2.GaussianBlur(img, (5, 5), 0)

  return result

inbreast_processed = []
for img in inbreast:
  inbreast_processed.append(preprocess(inbreast[0].pixel_array))

print('Finished preprocessing of INBreast data.')

Finished preprocessing of INBreast data.


In [0]:
def add_conv_layer(model, num_filters, dropout=None, pooling=True):
  model.add(K.layers.Conv2D(num_filters, (3,3), strides=(1,1), padding='same'))
  model.add(K.layers.Activation('relu'))
  model.add(K.layers.BatchNormalization())
  if pooling:
    model.add(K.layers.MaxPooling2D(2, 2))
  if dropout is not None:
    model.add(K.layers.Dropout(dropout))

def create_nrea_model(dropout):
  model = K.Sequential()
  add_conv_layer(model, 64, dropout)
  add_conv_layer(model, 128, None, False)
  add_conv_layer(model, 256, dropout)
  add_conv_layer(model, 512, dropout)
  add_conv_layer(model, 512, dropout)
  model.add(K.layers.Flatten())
  model.add(K.layers.Dense(2))
  model.add(K.Activation('relu'))